#**Importing required libraries**

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
!pip install split-folders
import splitfolders as sf
import os

In [ ]:
import time
import torch
import copy
from PIL import Image
from torch import nn, optim
from torchvision import transforms, models

In [ ]:
!pip install wandb
import wandb
wandb.login()
import gc

#**Downloading inatuaralist dataset zip file from drive**

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = '19EA0yl7PM8i6aQTdhH0OiEyLcTsf6hmx'

In [ ]:
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('nature_12K.zip')

#**Unzipping the content and distributing in train, validation and test folders**

In [ ]:
!apt install unzip

In [ ]:
!unzip 'nature_12K.zip'

In [ ]:
road='/content/inaturalist_12K/'
roadtrn=road+"train"
op= road+"trainvalsplit"
sf.fixed(roadtrn, op, seed=1337, fixed=100, oversample=False, group_prefix=None)

#**Preprocessing the images**

In [ ]:
train_data = []
train_label = []
path = '/content/inaturalist_12K/trainvalsplit/train/'
items = os.listdir(path)
items.sort()

data_augmentation = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


for i in range(10):
    image_folder_path = path + items[i]
    image_names = os.listdir(image_folder_path)
    for each_image in image_names:
        if each_image.endswith(".jpg"):
            full_path = image_folder_path + '/' + each_image
            image = Image.open(full_path)
            image = image.resize((224,224))
            if image.mode == 'L':
                continue
            normalized_image = data_augmentation(image)
            train_data.append((normalized_image, i))

In [ ]:
val_data = []
val_label = []
path = '/content/inaturalist_12K/trainvalsplit/val/'
items = os.listdir(path)
items.sort()

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

for i in range(10):
    image_folder_path = path + items[i]
    image_names = os.listdir(image_folder_path)
    for each_image in image_names:
        if each_image.endswith(".jpg"):
            full_path = image_folder_path + '/' + each_image
            image = Image.open(full_path)
            image = image.resize((224,224))
            if image.mode == 'L':
                continue
            normalized_image = transform(image)
            val_data.append((normalized_image, i))

In [ ]:
test_data = []
test_label = []
path = '/content/inaturalist_12K/val/'
items = os.listdir(path)
items.sort()

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

for i in range(10):
    image_folder_path = path + items[i]
    image_names = os.listdir(image_folder_path)
    for each_image in image_names:
        if each_image.endswith(".jpg"):
            full_path = image_folder_path + '/' + each_image
            image = Image.open(full_path)
            image = image.resize((224,224))
            if image.mode == 'L':
                continue
            normalized_image = transform(image)
            test_data.append((normalized_image, i))

In [ ]:
classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 
           'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#**Multiple Pre-Trained models**

In [ ]:
# # Resnet50 model
def resnet(freeze_percent):
    counttotal = 0
    resnet50_model = models.resnet50(pretrained=True)
    percent_of_layers_freezed = freeze_percent #0.25
    for param in resnet50_model.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in resnet50_model.parameters():
        if count<int(percent_of_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    num_features = resnet50_model.fc.in_features
    resnet50_model.fc = nn.Linear(num_features, 10)

    return resnet50_model

In [ ]:
# # vgg16 model
def vgg(freeze_percent):
    vgg16_model = models.vgg16_bn(pretrained=True)

    counttotal = 0
    percent_of_layers_freezed = freeze_percent #0.5
    for param in vgg16_model.features.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in vgg16_model.features.parameters():
        if count<int(percent_of_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    num_features = vgg16_model.classifier[6].in_features
    features = list(vgg16_model.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, len(classes))])
    vgg16_model.classifier = nn.Sequential(*features)

    return vgg16_model

In [ ]:
# # Alexnet model
def alexnet(freeze_percent):
    alexnet_model = models.alexnet(pretrained=True)
    counttotal = 0
    percent_of_layers_freezed = freeze_percent #0.25
    for param in alexnet_model.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in alexnet_model.parameters():
        if count<int(percent_of_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    alexnet_model.classifier[6] = nn.Linear(4096,10)

    return alexnet_model

In [ ]:
# # Squeezenet model
def squeezenet(freeze_percent):
    squeezenet_model = models.squeezenet1_1(pretrained=True)
    counttotal = 0
    percent_of_layers_freezed = freeze_percent #0.25
    for param in squeezenet_model.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in squeezenet_model.parameters():
        if count<int(percent_of_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    squeezenet_model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))

    return squeezenet_model

In [ ]:
# # densenet model
def densenet(freeze_percent):
    densenet_model = models.densenet161(pretrained=True)
    counttotal = 0
    percent_of_layers_freezed = freeze_percent #0.75
    for param in densenet_model.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in densenet_model.parameters():
        if count<int(percent_of_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    num_features = densenet_model.classifier.in_features
    densenet_model.classifier = nn.Linear(num_features, 10)

    return densenet_model

#**Defining training function and data loaders**

In [ ]:
def train_model(model, criteria, optimizer, train_loader, num_epochs=5, device='cuda', sweep=False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        # print('Epoch {}/{}'.format(epoch, num_epochs ))
        # print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.

            if phase == 'train':
                f = train_loader
            else:
                f = val_loader
            for inputs, labels in f:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = 0
            epoch_acc = 0
            if phase == 'train':
                epoch_loss = running_loss / len(train_data)
                epoch_acc = running_corrects.double() /len(train_data)
            else:
                epoch_loss = running_loss / len(val_data)
                epoch_acc = running_corrects.double() / len(val_data)
            epoch_acc*=100
            # print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        if sweep:
          wandb.log({'epoch' : epoch, 'val_accuracy' : best_acc})
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best validation acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    #return model

In [27]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=200, shuffle=True)

#**Training the models**

In [ ]:
# num_epochs = 10
fp = 0.25
# model = resnet(fp).to(device)
# # model = vgg(fp).to(device)
# # model = alexnet(fp).to(device)
# # model = squeezenet(fp).to(device)
# # model = densenet(fp).to(device)
# criteria = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.96,0.999))
# train_model(model, criteria, optimizer, train_loader, num_epochs, 'cuda')

#**Hyperparameter tuning**

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
    'parameters': {'model': {'values': ['densenet', 'resnet', 'vgg16', 'squeezenet', 'alexnet']},
                'learning_rate': {'values': [0.0001, 0.0003]},
                'beta1': {'values': [0.9, 0.92, 0.96]},
                'freeze_percent': {'values': [0.25, 0.5, 0.75]},
                'batch_size': {'values': [25, 50, 100]}
                }}

In [ ]:
def train():
    var1 = wandb.init()
    var2 = var1.config
        
    if var2.model == 'densenet':
        model = densenet(var2.freeze_percent).to(device)
    if var2.model == 'resnet':
        model = resnet(var2.freeze_percent).to(device)
    if var2.model == 'vgg16':
        model = vgg(var2.freeze_percent).to(device)
    if var2.model == 'squeezenet':
        model = squeezenet(var2.freeze_percent).to(device)
    if var2.model == 'alexnet':
        model = alexnet(var2.freeze_percent).to(device)


    train_loader = torch.utils.data.DataLoader(train_data, batch_size=var2.batch_size, shuffle=True)

    num_epochs = 5
    criteria = nn.CrossEntropyLoss() 
    optimizer = optim.Adam(model.parameters(), lr=var2.learning_rate, betas=(var2.beta1, 0.999))
    train_model(model, criteria, optimizer, train_loader, num_epochs, 'cuda', True)
   
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 2")
wandb.agent(sweep_id, train, count=25)#id: ty1meqo4

#**Evaluation on test data**

In [25]:
num_epochs = 10
fp = 0.25
# model = resnet(fp).to(device)
model = vgg(fp).to(device)
# model = alexnet(fp).to(device)
# model = squeezenet(fp).to(device)
# model = densenet(fp).to(device)

criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.96,0.999))
train_model(model, criteria, optimizer, train_loader, num_epochs, 'cuda')

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth



Training complete in 18m 59s
Best validation acc: 78.778779


In [24]:
def evaluate():
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(10)]
        n_class_samples = [0 for i in range(10)]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            
            for i in range(200):
                label = labels[i]
                pred = predicted[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc} %')

        for i in range(10):
            acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {classes[i]}: {acc} %')

In [28]:
evaluate()

Accuracy of the network: 79.3 %
Accuracy of Amphibia: 79.0 %
Accuracy of Animalia: 80.0 %
Accuracy of Arachnida: 75.5 %
Accuracy of Aves: 88.5 %
Accuracy of Fungi: 85.5 %
Accuracy of Insecta: 75.5 %
Accuracy of Mammalia: 85.5 %
Accuracy of Mollusca: 67.5 %
Accuracy of Plantae: 81.5 %
Accuracy of Reptilia: 74.5 %
